### pre-process the IRI-DL MME hindcasts ... 

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
from datetime import datetime, timedelta

In [12]:
import pygrib
import numpy as np
import pandas as pd

In [13]:
import xarray as xr; print(xr.__version__)

0.14.1


In [14]:
import xesmf as xe

In [15]:
import pathlib

In [16]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [17]:
HOME = pathlib.Path.home()

In [18]:
provider = 'IRI'

In [19]:
var_name = 'T2M'

In [20]:
GCM = 'NASA-GEOSS2S'

In [21]:
target_grid  = xr.open_dataset(HOME / 'research' / 'Smart_Ideas' / 'data' / 'GCMs' / 'NC' / 'target_grid' / 'target_grid_2.5x2.5.nc') 

In [22]:
def regrid(ds_in, target_ds,  method='bilinear', clean=False):
    import xesmf as xe
    """Convenience function for one-time regridding"""
    regridder = xe.Regridder(ds_in, target_ds, method, periodic=True, reuse_weights=True)
    ds_out = regridder(ds_in)
    if clean: 
        regridder.clean_weight_file()
    return ds_out

In [23]:
def make_forecast_valid_time(dset, nsteps): 
    from dateutil.relativedelta import relativedelta
    import pandas as pd
    import numpy as np
    time = dset.time.to_series()
    tv = [x + relativedelta(months=m + 2) for m in range(1, nsteps + 1) for x in time]
    tv = np.array(tv)
    tv = tv.reshape((len(tv) // nsteps,nsteps))
    dset['forecast_valid_time'] = (('time','step'), tv)
    return dset

In [24]:
dpath = HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'hindcasts' / provider / GCM / var_name

In [25]:
dpath

PosixPath('/home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/IRI/NASA-GEOSS2S/T2M')

In [26]:
opath = HOME / 'research' / 'Smart_Ideas' / 'data' / 'GCMs' / 'NC' / GCM / var_name

In [27]:
opath

PosixPath('/home/nicolasf/research/Smart_Ideas/data/GCMs/NC/NASA-GEOSS2S/T2M')

In [28]:
if not opath.exists(): 
    opath.mkdir(parents=True)

In [29]:
lfiles = list(dpath.glob("*.nc"))

In [30]:
def sf(x): 
    x = x.name 
    x = x.split('_')[-2] + x.split('_')[-1].split('.nc')[0].zfill(2) + '01'
    return int(x)

In [31]:
def sfdates(x): 
    from datetime import datetime
    x = x.name 
    d = datetime(int(x.split('_')[-2]), int(x.split('_')[-1].split('.nc')[0]), 1)
    return d

In [32]:
lfiles.sort(key=sf)

In [33]:
len(lfiles)

0

In [34]:
print(lfiles[0])

IndexError: list index out of range

In [35]:
print(lfiles[-1])

IndexError: list index out of range

In [69]:
for fname in lfiles: 
    year = fname.name.split('_')[-2]
    month = fname.name.split('_')[-1].split('.')[0] 
    
    fname_out = f'{var_name}_hindcasts_{year}_{month}_{GCM}_seasonal_interp.nc' 
    
    print(fname_out)
    
    dset = xr.open_dataset(fname)
    
    dset = dset.mean('member')
    
    dset = regrid(dset, target_grid) 
    
    dset['step'] -= 0.5 
    
    dset.step.data = dset.step.data.astype(np.int) 
    
    dset = dset.rolling({'step':3}, min_periods=3).mean()
    
    dset = dset.sel(step=slice(3,5))
    
    dset['time'] = ('time', dset['time'].data.reshape(-1, )) 
    
    dset[var_name.lower()] = (('time','step','lat','lon'), dset[var_name.lower()].data.reshape((-1, *dset[var_name.lower()].data.shape)))
    
    dset = make_forecast_valid_time(dset, 3)
    
    dset.to_netcdf(opath / fname_out)
    
    dset.close()

T2M_hindcasts_1981_1_CanSIPSv2_seasonal_interp.nc
Reuse existing file: bilinear_181x360_73x144_peri.nc
using dimensions ('lat', 'lon') from data variable t2m as the horizontal dimensions for this dataset.
T2M_hindcasts_1981_2_CanSIPSv2_seasonal_interp.nc
Reuse existing file: bilinear_181x360_73x144_peri.nc
using dimensions ('lat', 'lon') from data variable t2m as the horizontal dimensions for this dataset.
T2M_hindcasts_1981_3_CanSIPSv2_seasonal_interp.nc
Reuse existing file: bilinear_181x360_73x144_peri.nc
using dimensions ('lat', 'lon') from data variable t2m as the horizontal dimensions for this dataset.
T2M_hindcasts_1981_4_CanSIPSv2_seasonal_interp.nc
Reuse existing file: bilinear_181x360_73x144_peri.nc
using dimensions ('lat', 'lon') from data variable t2m as the horizontal dimensions for this dataset.
T2M_hindcasts_1981_5_CanSIPSv2_seasonal_interp.nc
Reuse existing file: bilinear_181x360_73x144_peri.nc
using dimensions ('lat', 'lon') from data variable t2m as the horizontal dime

OSError: [Errno -101] NetCDF: HDF error: b'/home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/IRI/CanSIPSv2/T2M/CFSv2_hindcasts_T2M_1998_9.nc'